# Importing Library

In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
df = pd.read_csv('GroceryStoreDataSet (2).csv')
df

,ITEMS
0,"MILK,BREAD,BISCUIT"
1,"BREAD,MILK,BISCUIT,CORNFLAKES"
2,"BREAD,TEA,BOURNVITA"
3,"JAM,MAGGI,BREAD,MILK"
4,"MAGGI,TEA,BISCUIT"
5,"BREAD,TEA,BOURNVITA"
6,"MAGGI,TEA,CORNFLAKES"
7,"MAGGI,BREAD,TEA,BISCUIT"
8,"JAM,MAGGI,BREAD,TEA"
9,"BREAD,MILK"


In [ ]:
df.shape

(20, 1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ITEMS   20 non-null     object
dtypes: object(1)
memory usage: 288.0+ bytes


In [ ]:
df.index.rename('TID', inplace=True)


In [ ]:
trans_df = df.ITEMS.str.split(',')
trans_df

TID
0                  [MILK, BREAD, BISCUIT]
1      [BREAD, MILK, BISCUIT, CORNFLAKES]
2                 [BREAD, TEA, BOURNVITA]
3               [JAM, MAGGI, BREAD, MILK]
4                   [MAGGI, TEA, BISCUIT]
5                 [BREAD, TEA, BOURNVITA]
6                [MAGGI, TEA, CORNFLAKES]
7            [MAGGI, BREAD, TEA, BISCUIT]
8                [JAM, MAGGI, BREAD, TEA]
9                           [BREAD, MILK]
10    [COFFEE, COCK, BISCUIT, CORNFLAKES]
11    [COFFEE, COCK, BISCUIT, CORNFLAKES]
12             [COFFEE, SUGER, BOURNVITA]
13                  [BREAD, COFFEE, COCK]
14                [BREAD, SUGER, BISCUIT]
15            [COFFEE, SUGER, CORNFLAKES]
16              [BREAD, SUGER, BOURNVITA]
17                 [BREAD, COFFEE, SUGER]
18                 [BREAD, COFFEE, SUGER]
19        [TEA, MILK, COFFEE, CORNFLAKES]
Name: ITEMS, dtype: object

In [ ]:
def prune(data,supp):
    
    df = data[data.supp_count >= supp] 
    return df

In [ ]:
def count_itemset(transaction_df, itemsets):
    
    count_item = {}
    for item_set in itemsets:
        set_A = set(item_set)
        for row in trans_df:
            set_B = set(row)
            if set_B.intersection(set_A) == set_A: 
                if item_set in count_item.keys():
                    count_item[item_set] += 1
                
                else:
                    count_item[item_set] = 1
                
    data = pd.DataFrame()
    data['item_sets'] = count_item.keys()
    data['supp_count'] = count_item.values()
    
    return data

# here we take minimum support count is 3

In [ ]:
def apriori(trans_data,supp=3, con=0.5):
    freq = pd.DataFrame()
    
    df = count_item(trans_data)
   
    while(len(df) != 0):
        
        df = prune(df, supp)
    
        if len(df) > 1 or (len(df) == 1 and int(df.supp_count >= supp)):
            freq = df
        
        itemsets = join(df.item_sets)
        if(itemsets is None):
            return freq
    
        df = count_itemset(trans_data, itemsets)
    return df

In [ ]:
def count_item(trans_items):
    
    count_ind_item = {}
    for row in trans_items:
        for i in range(len(row)):
            if row[i] in count_ind_item.keys():
                count_ind_item[row[i]] += 1
            else:
                count_ind_item[row[i]] = 1
                data = pd.DataFrame()
    data['item_sets'] = count_ind_item.keys()
    data['supp_count'] = count_ind_item.values()
    data = data.sort_values('item_sets')
    return data

In [ ]:
def join(list_of_items):
    itemsets = []
    i = 1
    for entry in list_of_items:
        proceding_items = list_of_items[i:]
        for item in proceding_items:
            if(type(item) is str):
                if entry != item:
                    tuples = (entry, item)
                    itemsets.append(tuples)
            else:
                if entry[0:-1] == item[0:-1]:
                    tuples = entry+item[1:]
                    itemsets.append(tuples)
        i = i+1
    if(len(itemsets) == 0):
        return None
    return itemsets

In [ ]:
freq_item_sets = apriori(trans_df, 4)
freq_item_sets

,item_sets,supp_count
0,"(BISCUIT, BREAD)",4
14,"(BREAD, MILK)",4
15,"(BREAD, SUGER)",4
16,"(BREAD, TEA)",4
17,"(COFFEE, CORNFLAKES)",4
19,"(COFFEE, SUGER)",4
26,"(MAGGI, TEA)",4


In [ ]:
def calculate_conf(value1, value2):
    return round(int(value1)/int(value2) * 100, 2)

In [ ]:
def strong_rules(freq_item_sets, threshold):

    confidences = {}
    for row in freq_item_sets.item_sets:
        for i in range(len(row)):
            for j in range(len(row)):
                 if i != j:
                    tuples = (row[i], row[j])
                    conf = calculate_conf(freq_item_sets[freq_item_sets.item_sets == row].supp_count, count_item(trans_df)[count_item(trans_df).item_sets == row[i]].supp_count)
                    confidences[tuples] = conf
                    conf_df = pd.DataFrame()
    conf_df['item_set'] = confidences.keys()
    conf_df['confidence'] = confidences.values()

    return conf_df[conf_df.confidence >= threshold]

In [ ]:
strong_rules(freq_item_sets, 40.0)

,item_set,confidence
0,"(BISCUIT, BREAD)",57.14
3,"(MILK, BREAD)",80.00
5,"(SUGER, BREAD)",66.67
7,"(TEA, BREAD)",57.14
8,"(COFFEE, CORNFLAKES)",50.00
9,"(CORNFLAKES, COFFEE)",66.67
10,"(COFFEE, SUGER)",50.00
11,"(SUGER, COFFEE)",66.67
12,"(MAGGI, TEA)",80.00
13,"(TEA, MAGGI)",57.14


In [ ]:
from functools import reduce # Valid in Python 2.6+, required in Python 3
import operator

def interesting_rules(freq_item_sets):
    
    lifts = {}
    prob_of_items = []
  
    for row in freq_item_sets.item_sets:
        num_of_items = len(row)
        
        prob_all = freq_item_sets[freq_item_sets.item_sets == row].supp_count / 18
        for i in range(num_of_items):
            prob_of_items.append(count_item(trans_df)[count_item(trans_df).item_sets == row[i]].supp_count / 18)

In [ ]:
def strong_rules(freq_item_sets, threshold):

    confidences = {}
    for row in freq_item_sets.item_sets:
        for i in range(len(row)):
            for j in range(len(row)):
                 if i != j:
                    tuples = (row[i], row[j])
                    conf = calculate_conf(freq_item_sets[freq_item_sets.item_sets == row].supp_count, count_item(trans_df)[count_item(trans_df).item_sets == row[i]].supp_count)
                    confidences[tuples] = conf
    conf_df = pd.DataFrame()
    conf_df['item_set'] = confidences.keys()
    conf_df['confidence'] = confidences.values()

    return conf_df[conf_df.confidence >= threshold]

In [ ]:
strong_rules(freq_item_sets, 50.0)

,item_set,confidence
0,"(BISCUIT, BREAD)",57.14
3,"(MILK, BREAD)",80.00
5,"(SUGER, BREAD)",66.67
7,"(TEA, BREAD)",57.14
8,"(COFFEE, CORNFLAKES)",50.00
9,"(CORNFLAKES, COFFEE)",66.67
10,"(COFFEE, SUGER)",50.00
11,"(SUGER, COFFEE)",66.67
12,"(MAGGI, TEA)",80.00
13,"(TEA, MAGGI)",57.14
